 # Classification des avis sur des vêtements de femmes vendus dans le e-commerce

- est ce que les avis que l'on a des vêtements est représentatif de la note qui est attribué ?

idées 
- visualisation de données : quels types de vêtements ont les notes les plus élevées ?
- nb d'avis donné selon l'âge des clients

In [34]:
import pandas as pd

## I. Import des données

In [35]:
data = pd.read_csv("Womens Clothing E-Commerce Reviews.csv", sep = ",")
data.head()

,Unnamed: 0,Clothing ID,Age,Title,Review Text,Rating,Recommended IND,Positive Feedback Count,Division Name,Department Name,Class Name
0,0,767,33,NaN,Absolutely wonderful - silky and sexy and comf...,4,1,0,Initmates,Intimate,Intimates
1,1,1080,34,NaN,Love this dress! it's sooo pretty. i happene...,5,1,4,General,Dresses,Dresses
2,2,1077,60,Some major design flaws,I had such high hopes for this dress and reall...,3,0,0,General,Dresses,Dresses
3,3,1049,50,My favorite buy!,"I love, love, love this jumpsuit. it's fun, fl...",5,1,0,General Petite,Bottoms,Pants
4,4,847,47,Flattering shirt,This shirt is very flattering to all due to th...,5,1,6,General,Tops,Blouses


In [36]:
nb_dpt = data.groupby("Department Name").count()
nb_dpt

,Unnamed: 0,Clothing ID,Age,Title,Review Text,Rating,Recommended IND,Positive Feedback Count,Division Name,Class Name
Department Name,,,,,,,,,,
Bottoms,3799,3799,3799,3184,3662,3799,3799,3799,3799,3799
Dresses,6319,6319,6319,5372,6145,6319,6319,6319,6319,6319
Intimate,1735,1735,1735,1408,1653,1735,1735,1735,1735,1735
Jackets,1032,1032,1032,879,1002,1032,1032,1032,1032,1032
Tops,10468,10468,10468,8713,10048,10468,10468,10468,10468,10468
Trend,119,119,119,107,118,119,119,119,119,119


### Récupération des données pour notre étude

### Traitement de la casse

- suppression des valeurs manquantes => pas d'avis : inutile
- suppression des caractères spéciaux 
- suppression des majuscules
- suppression des mots vides
- lemmatisation 
- affiche du nombre de mots par étiquette grammaticale
- extraction des mots (groupes de mots) les plus fréquents

- wordcloud

Dans un premier temps, nous allons récupérer les avis.

In [37]:
avis = data["Review Text"]
avis

0        Absolutely wonderful - silky and sexy and comf...
1        Love this dress!  it's sooo pretty.  i happene...
2        I had such high hopes for this dress and reall...
3        I love, love, love this jumpsuit. it's fun, fl...
4        This shirt is very flattering to all due to th...
                               ...                        
23481    I was very happy to snag this dress at such a ...
23482    It reminds me of maternity clothes. soft, stre...
23483    This fit well, but the top was very see throug...
23484    I bought this dress for a wedding i have this ...
23485    This dress in a lovely platinum is feminine an...
Name: Review Text, Length: 23486, dtype: object

Nous allons récupérer seulement la partie textuelle de l'avis, cela nous permet de ne avoir un objet Pandas.Series.

In [38]:
from pprint import pprint
liste_avis = data["Review Text"].values.tolist()
pprint(liste_avis[:10])

['Absolutely wonderful - silky and sexy and comfortable',
 "Love this dress!  it's sooo pretty.  i happened to find it in a store, and "
 "i'm glad i did bc i never would have ordered it online bc it's petite.  i "
 'bought a petite and am 5\'8".  i love the length on me- hits just a little '
 'below the knee.  would definitely be a true midi on someone who is truly '
 'petite.',
 'I had such high hopes for this dress and really wanted it to work for me. i '
 'initially ordered the petite small (my usual size) but i found this to be '
 'outrageously small. so small in fact that i could not zip it up! i reordered '
 'it in petite medium, which was just ok. overall, the top half was '
 'comfortable and fit nicely, but the bottom half had a very tight under layer '
 'and several somewhat cheap (net) over layers. imo, a major design flaw was '
 'the net over layer sewn directly into the zipper - it c',
 "I love, love, love this jumpsuit. it's fun, flirty, and fabulous! every time "
 'i wea

Grâce à cette manipulation, chaque avis est élément d'une liste d'avis. 

Ensuite, nous allons découper les avis en liste de mots et les mettre en minuscules pour pouvoir les analyser plus facilement.

In [39]:
liste_avis_clean = []

for avis in liste_avis : 
    avis = str(avis)
    avis_clean = avis.split()
    avis_clean = avis.lower()
    liste_avis_clean.append(avis_clean)

liste_avis_clean[:10]

['absolutely wonderful - silky and sexy and comfortable',
 'love this dress!  it\'s sooo pretty.  i happened to find it in a store, and i\'m glad i did bc i never would have ordered it online bc it\'s petite.  i bought a petite and am 5\'8".  i love the length on me- hits just a little below the knee.  would definitely be a true midi on someone who is truly petite.',
 'i had such high hopes for this dress and really wanted it to work for me. i initially ordered the petite small (my usual size) but i found this to be outrageously small. so small in fact that i could not zip it up! i reordered it in petite medium, which was just ok. overall, the top half was comfortable and fit nicely, but the bottom half had a very tight under layer and several somewhat cheap (net) over layers. imo, a major design flaw was the net over layer sewn directly into the zipper - it c',
 "i love, love, love this jumpsuit. it's fun, flirty, and fabulous! every time i wear it, i get nothing but great compliments

Puis, nous allons tockeniser notre texte. 

In [40]:
from nltk.tokenize import RegexpTokenizer

tokenizer = RegexpTokenizer('\w+')
liste_avis_clean = [tokenizer.tokenize(str(avis)) for avis in liste_avis_clean]
liste_avis_clean[:10]

[['absolutely', 'wonderful', 'silky', 'and', 'sexy', 'and', 'comfortable'],
 ['love',
  'this',
  'dress',
  'it',
  's',
  'sooo',
  'pretty',
  'i',
  'happened',
  'to',
  'find',
  'it',
  'in',
  'a',
  'store',
  'and',
  'i',
  'm',
  'glad',
  'i',
  'did',
  'bc',
  'i',
  'never',
  'would',
  'have',
  'ordered',
  'it',
  'online',
  'bc',
  'it',
  's',
  'petite',
  'i',
  'bought',
  'a',
  'petite',
  'and',
  'am',
  '5',
  '8',
  'i',
  'love',
  'the',
  'length',
  'on',
  'me',
  'hits',
  'just',
  'a',
  'little',
  'below',
  'the',
  'knee',
  'would',
  'definitely',
  'be',
  'a',
  'true',
  'midi',
  'on',
  'someone',
  'who',
  'is',
  'truly',
  'petite'],
 ['i',
  'had',
  'such',
  'high',
  'hopes',
  'for',
  'this',
  'dress',
  'and',
  'really',
  'wanted',
  'it',
  'to',
  'work',
  'for',
  'me',
  'i',
  'initially',
  'ordered',
  'the',
  'petite',
  'small',
  'my',
  'usual',
  'size',
  'but',
  'i',
  'found',
  'this',
  'to',
  'be',
 

Ensuite, on supprime la ponctuation.

In [41]:
import string
punct = string.punctuation

# Pour chaque token dans chaque avis, si le token n'est pas dans la liste des ponctuations, on le garde
liste_avis_clean = [[token for token in avis if token not in punct] for avis in liste_avis_clean]
liste_avis_clean[:10]

[['absolutely', 'wonderful', 'silky', 'and', 'sexy', 'and', 'comfortable'],
 ['love',
  'this',
  'dress',
  'it',
  's',
  'sooo',
  'pretty',
  'i',
  'happened',
  'to',
  'find',
  'it',
  'in',
  'a',
  'store',
  'and',
  'i',
  'm',
  'glad',
  'i',
  'did',
  'bc',
  'i',
  'never',
  'would',
  'have',
  'ordered',
  'it',
  'online',
  'bc',
  'it',
  's',
  'petite',
  'i',
  'bought',
  'a',
  'petite',
  'and',
  'am',
  '5',
  '8',
  'i',
  'love',
  'the',
  'length',
  'on',
  'me',
  'hits',
  'just',
  'a',
  'little',
  'below',
  'the',
  'knee',
  'would',
  'definitely',
  'be',
  'a',
  'true',
  'midi',
  'on',
  'someone',
  'who',
  'is',
  'truly',
  'petite'],
 ['i',
  'had',
  'such',
  'high',
  'hopes',
  'for',
  'this',
  'dress',
  'and',
  'really',
  'wanted',
  'it',
  'to',
  'work',
  'for',
  'me',
  'i',
  'initially',
  'ordered',
  'the',
  'petite',
  'small',
  'my',
  'usual',
  'size',
  'but',
  'i',
  'found',
  'this',
  'to',
  'be',
 

## II. Pré-traitement des données

## III. Méthode de fouille de textes